<a href="https://colab.research.google.com/github/Joanmei/Engage-Project/blob/main/Karatina_project_using_k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the dataset
df = pd.read_excel('/content/drive/MyDrive/Karatina Project/CLIENTS LIST.xlsx')
# View the first few rows of the DataFrame
print(df.head())

   Weight  Height  ht in MTRS        BMI Blood Pressure  Age at Reporting Sex  \
0    64.0   165.0       1.650  23.507805         118/73                51   F   
1    56.4   168.8       1.688  19.794030         133/82                54   M   
2    73.0   160.0       1.600  28.515625         104/57                54   F   
3    62.0   161.0       1.610  23.918830         108/68                45   F   
4    90.0   175.0       1.750  29.387755         126/88                34   F   

          Popolation Type Date confirmed HIV Positive Date enrolled in HIV  \
0  General Population\n\n                  2014-06-10  2014-08-04 00:00:00   
1  General Population\n\n                  2012-09-04  2014-03-24 00:00:00   
2  General Population\n\n                  2013-07-11  2013-07-11 00:00:00   
3  General Population\n\n                  2014-04-21  2014-04-21 00:00:00   
4  General Population\n\n                  2013-07-16  2013-07-16 00:00:00   

   ... TPT Outcome Date Establishment Differ

In [4]:
# Check for missing values
print(df.isna().sum())

Weight                            0
Height                            0
ht in MTRS                        0
BMI                               0
Blood Pressure                   34
Age at Reporting                  0
Sex                               0
Popolation Type                   0
Date confirmed HIV Positive       0
Date enrolled in HIV              0
ART Start Date                    0
First Regimen                     0
Current Regimen                   0
Baseline CD4 result             644
Baseline CD4 date               644
Current Regimen Line              0
Last WHO Stage                    0
Last WHO Stage Date               0
Last VL                          24
Last VL Validity                 24
Last VL Justification            37
Last VL Date                     24
Active in PMTCT                   0
Active in OVC                     0
Active in OTZ                     0
Active in TB                      0
TPT Start Date                   58
TPT Outcome                 

In [5]:
# Drop columns with too many missing values
df = df.drop(columns=['Medical Cover'])
# Fill missing values for numeric columns with median (for example: 'Blood Pressure', 'Last VL')

# Split the 'Blood Pressure' column into systolic and diastolic
df[['Systolic', 'Diastolic']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)

# Calculate the median for Systolic and Diastolic separately
df['Systolic'].fillna(df['Systolic'].median(), inplace=True)
df['Diastolic'].fillna(df['Diastolic'].median(), inplace=True)

# Combine the Systolic and Diastolic columns back to 'Blood Pressure' in the desired format
df['Blood Pressure'] = df['Systolic'].astype(str) + '/' + df['Diastolic'].astype(str)

df['Last VL'].fillna(df['Last VL'].median(), inplace=True)

In [6]:
# Fill missing values for date columns using forward fill or backward fill
df['Baseline CD4 date'].fillna(method='ffill', inplace=True)
df['TPT Outcome Date'].fillna(method='bfill', inplace=True)

<ipython-input-6-c840372d844e>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Baseline CD4 date'].fillna(method='ffill', inplace=True)
<ipython-input-6-c840372d844e>:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['TPT Outcome Date'].fillna(method='bfill', inplace=True)


In [7]:
# For categorical columns, you can fill missing values with the most frequent value
df['Last VL Justification'].fillna(df['Last VL Justification'].mode()[0], inplace=True)
# Drop rows where critical values like CD4 counts or visit dates are missing
df = df.dropna(subset=['Baseline CD4 result', 'Last Visit Date'])

In [8]:
#Check again for missing values
print(df.isnull().sum())

Weight                           0
Height                           0
ht in MTRS                       0
BMI                              0
Blood Pressure                   0
Age at Reporting                 0
Sex                              0
Popolation Type                  0
Date confirmed HIV Positive      0
Date enrolled in HIV             0
ART Start Date                   0
First Regimen                    0
Current Regimen                  0
Baseline CD4 result              0
Baseline CD4 date                0
Current Regimen Line             0
Last WHO Stage                   0
Last WHO Stage Date              0
Last VL                          0
Last VL Validity                 6
Last VL Justification            0
Last VL Date                     6
Active in PMTCT                  0
Active in OVC                    0
Active in OTZ                    0
Active in TB                     0
TPT Start Date                  36
TPT Outcome                     46
TPT Outcome Date    

In [9]:
print(df.shape)

(1922, 39)


In [10]:
# Save cleaned data
df.to_excel("cleaned_data.xlsx", index=False)

In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Separate numerical and categorical variables
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

print("Numerical Columns:", numerical_cols)
print("Categorical Columns:", categorical_cols)

Numerical Columns: Index(['Weight', 'Height', 'ht in MTRS', 'BMI', 'Age at Reporting',
       'Last WHO Stage', 'Last VL', 'Months Of Prescription', 'Systolic',
       'Diastolic'],
      dtype='object')
Categorical Columns: Index(['Blood Pressure', 'Sex', 'Popolation Type', 'Date enrolled in HIV',
       'First Regimen', 'Current Regimen', 'Baseline CD4 result',
       'Baseline CD4 date', 'Current Regimen Line', 'Last VL Validity',
       'Last VL Justification', 'Active in PMTCT', 'Active in OVC',
       'Active in OTZ', 'Active in TB', 'TPT Outcome', 'Establishment',
       'Differentiated Care Model', 'AHD Client'],
      dtype='object')


In [12]:
# Handle categorical encoding
# Only 'Sex', 'Population Type', 'Current Regimen Line'
label_encoder = LabelEncoder()

df['Sex'] = label_encoder.fit_transform(df['Sex'])
df['Popolation Type'] = label_encoder.fit_transform(df['Popolation Type'])
df['Current Regimen Line'] = label_encoder.fit_transform(df['Current Regimen Line'])

In [13]:
df.head()

,Weight,Height,ht in MTRS,BMI,Blood Pressure,Age at Reporting,Sex,Popolation Type,Date confirmed HIV Positive,Date enrolled in HIV,...,Establishment,Differentiated Care Model,Last Visit Date,Self Visit Date,Next Appointment Date,Months Of Prescription,Refill Date,AHD Client,Systolic,Diastolic
2,73.0,160.0,1.600,28.515625,104.0/57.0,54,0,1,2013-07-11,2013-07-11 00:00:00,...,Established,Fast Track,2024-08-30,2024-08-30,2025-02-27,5,2024-11-28,Yes,104.0,57.0
4,90.0,175.0,1.750,29.387755,126.0/88.0,34,0,1,2013-07-16,2013-07-16 00:00:00,...,Established,Fast Track,2024-05-02,2024-05-02,2024-10-31,5,2024-08-01,No,126.0,88.0
5,73.0,157.0,1.570,29.615806,115.0/83.0,31,0,1,2013-07-16,2013-07-16 00:00:00,...,Established,Fast Track,2024-06-19,2024-06-19,2025-01-14,6,2024-09-18,No,115.0,83.0
6,90.0,159.9,1.599,35.200237,102.0/65.0,42,0,0,2013-09-17,2013-09-17 00:00:00,...,Established,Standard Care,2024-06-04,2024-03-07,2024-09-03,2,2023-06-07,Yes,102.0,65.0
8,62.2,174.0,1.740,20.544326,94.0/65.0,48,1,1,2008-06-15,2013-07-15 00:00:00,...,Not Established,Standard Care,2024-07-03,2024-07-03,2024-10-02,2,2021-03-11,No,94.0,65.0


In [21]:
from sklearn.preprocessing import StandardScaler
continuous_cols = ['Weight', 'Height','Blood Pressure', 'BMI', 'Age at Reporting']
# Handle 'Blood Pressure' column
df[['Systolic', 'Diastolic']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)

# Update continuous_cols to include new columns and remove 'Blood Pressure'
continuous_cols = ['Weight', 'Height', 'Systolic', 'Diastolic', 'BMI', 'Age at Reporting']

# Now scale the data
scaler = StandardScaler()
df[continuous_cols] = scaler.fit_transform(df[continuous_cols])

In [22]:
# Check the standardized data
print(df[continuous_cols].head())

     Weight    Height  Systolic  Diastolic       BMI  Age at Reporting
2  0.687485 -0.103844 -0.586998  -1.152144  0.776238          0.503237
4  1.916363  1.438565  0.266883   0.795953  0.944725         -0.941347
5  0.687485 -0.412326 -0.160057   0.481744  0.988782         -1.158035
6  1.916363 -0.114127 -0.664623  -0.649409  2.067638         -0.363514
8 -0.093213  1.335738 -0.975126  -0.649409 -0.763738          0.069862


In [24]:
# Save the standardized dataframe
df.to_excel('standardized_data.xlsx', index=False)

In [26]:
from sklearn.decomposition import PCA
df = pd.read_excel('/content/drive/MyDrive/Karatina Project/standardized_data.xlsx')
# Select key features for clustering
key_features = ['BMI', 'Systolic', 'Diastolic', 'Age at Reporting', 'Baseline CD4 result',
                'Last VL', 'ART Start Date', 'Current Regimen', 'Current Regimen Line',
                'Last WHO Stage', 'TPT Outcome']

# Subset the dataframe for selected features
df_selected = df[key_features]

# Standardize the continuous variables
scaler = StandardScaler()
df_standardized = scaler.fit_transform(df_selected)


ValueError: could not convert string to float: '>200'